In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np


1.Define a Convolutional Neural Network



In [9]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [10]:
def imshow(img):
  img = img / 2 + 0.5  # unnormalize
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()

In [11]:
def save_checkpoint(state, save_dir, filename='checkpoint.pth'):
  save_name = save_dir + '_{}'.format(filename)
  torch.save(state, save_name)

2.Define the CIFAR10 Dataset

In [12]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


net = Net()

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


3.
Define the Loss Function and optimizer

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

4.Train the network

In [14]:
for epoch in range(3):  # loop over the dataset multiple times

  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data


    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    # print statistics
    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
  
      print("Epoch : {} steps : {} Training Loss : {}".format(epoch + 1, i + 1, running_loss / 2000) )
      running_loss = 0.0
  save_checkpoint({'net':net.state_dict()}, 'test_epoch{}'.format(epoch+1))        

print('Finished Training')

Epoch : 1 steps : 2000 Training Loss : 2.1889700249433517
Epoch : 1 steps : 4000 Training Loss : 1.8390005451440812
Epoch : 1 steps : 6000 Training Loss : 1.643870963960886
Epoch : 1 steps : 8000 Training Loss : 1.5679095338881015
Epoch : 1 steps : 10000 Training Loss : 1.492728895097971
Epoch : 1 steps : 12000 Training Loss : 1.4820199556872249
Epoch : 2 steps : 2000 Training Loss : 1.407832405924797
Epoch : 2 steps : 4000 Training Loss : 1.3714569466710091
Epoch : 2 steps : 6000 Training Loss : 1.3651220782622695
Epoch : 2 steps : 8000 Training Loss : 1.3291044723726808
Epoch : 2 steps : 10000 Training Loss : 1.284140573028475
Epoch : 2 steps : 12000 Training Loss : 1.2962907982245087
Epoch : 3 steps : 2000 Training Loss : 1.2023748714327813
Epoch : 3 steps : 4000 Training Loss : 1.2107961179297417
Epoch : 3 steps : 6000 Training Loss : 1.1705148331448436
Epoch : 3 steps : 8000 Training Loss : 1.2100771391429006
Epoch : 3 steps : 10000 Training Loss : 1.185110432397574
Epoch : 3 step

In [15]:
dataiter = iter(testloader)
images, labels = dataiter.next()

In [16]:
dataiter = iter(testloader)
images, labels = dataiter.next()
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]for j in range(4)))
correct = 0
total = 0
with torch.no_grad():
  for data in testloader:
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy : %d %%' % (100 * correct / total))

Predicted:    cat   car   car  ship
Accuracy : 58 %
